In [1]:
import os
import tensorflow as tf
from datasets import flowers, dataset_utils
from nets import inception
from nets import alexnet
from preprocessing import inception_preprocessing
import matplotlib.pyplot as plt
import numpy as np

slim = tf.contrib.slim

def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.

    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.

    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])

    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)

    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
        [image, image_raw, label],
        batch_size=batch_size,
        num_threads=1,
        capacity=2 * batch_size)

    return images, images_raw, labels

flowers_data_dir = '/home/gao/Data/flower/tf_data/slim/flower_fine_tune/flowers'
# train_dir = '/tmp/inception_finetuned/'
train_dir = '/home/gao/Data/flower/tf_data/slim/inception_v1_fine_tune/flowers-models/inception_v1/all'

image_size = inception.inception_v1.default_image_size
batch_size = 5

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)

    # dataset = flowers.get_split('train', flowers_data_dir)
    dataset = flowers.get_split('validation', flowers_data_dir)
    images, images_raw, labels = load_batch(dataset, height=image_size, width=image_size)

    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=False)

    probabilities = tf.nn.softmax(logits)

    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())

    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
#             sess.run(tf.initialize_local_variables())
            init_fn(sess)
            np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])

            for i in xrange(batch_size):
                image = np_images_raw[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                predicted_name = dataset.labels_to_names[predicted_label]
                true_name = dataset.labels_to_names[true_label]

                plt.figure()
                plt.imshow(image.astype(np.uint8))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.axis('off')
                plt.show()

INFO:tensorflow:Restoring parameters from /home/gao/Data/flower/tf_data/slim/inception_v1_fine_tune/flowers-models/inception_v1/all/model.ckpt-1000
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, assertion failed: [More than one condition evaluated as True but exclusive=True.  Conditions: (LogicalOr:0, LogicalOr_1:0), Values:] [0 0]
	 [[Node: case/Assert/AssertGuard/Assert = Assert[T=[DT_STRING, DT_BOOL], summarize=2, _device="/job:localhost/replica:0/task:0/cpu:0"](case/Assert/AssertGuard/Assert/Switch/_29, case/Assert/AssertGuard/Assert/data_0, case/Assert/AssertGuard/Assert/Switch_1)]]

Caused by op u'case/Assert/AssertGuard/Assert', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", 

InvalidArgumentError: assertion failed: [More than one condition evaluated as True but exclusive=True.  Conditions: (LogicalOr:0, LogicalOr_1:0), Values:] [0 0]
	 [[Node: case/Assert/AssertGuard/Assert = Assert[T=[DT_STRING, DT_BOOL], summarize=2, _device="/job:localhost/replica:0/task:0/cpu:0"](case/Assert/AssertGuard/Assert/Switch/_29, case/Assert/AssertGuard/Assert/data_0, case/Assert/AssertGuard/Assert/Switch_1)]]

Caused by op u'case/Assert/AssertGuard/Assert', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py